In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train  = pd.read_csv('/content/drive/MyDrive/TP3-Orga/weakly_train.csv', usecols = ['text', 'target'])
df_test = pd.read_csv('/content/drive/MyDrive/TP3-Orga/labelled_test.csv', usecols = ['text', 'target'])

In [ ]:
X_train = df_train.drop("target", axis=1)
X_test = df_test.drop("target", axis=1)

y_train = df_train["target"]
y_test = df_test["target"]

In [ ]:
X_train = X_train['text']
X_test = X_test['text']

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow-text
import tensorflow_text as text

In [ ]:
from tensorflow.keras.utils import set_random_seed

In [ ]:
set_random_seed(10)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from tensorflow.keras.metrics import AUC

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[AUC(curve="ROC")])

In [ ]:
model.fit(X_train, y_train, epochs=1, batch_size = 50, validation_split=0.15)

195/195 [==============================] - 6038s 31s/step - loss: 0.5301 - auc: 0.7968 - val_loss: 0.7387 - val_auc: 0.8322


In [ ]:
preds = model.predict(X_test)

19/19 [==============================] - 312s 16s/step


In [ ]:
roc_auc_score(y_test, preds)

0.8947666666666666

In [ ]:
frases = ["i love cars", "that player is too bad", "he is ugly", "man", "women", "today it is a sunny day", "this car is shit", "love love love"]

In [ ]:
model.predict(frases)

1/1 [==============================] - 5s 5s/step


array([[0.52110636],
       [0.65304655],
       [0.62915796],
       [0.43072098],
       [0.36893827],
       [0.6606595 ],
       [0.68755513],
       [0.6499026 ]], dtype=float32)